In [1]:
import torch
from datasets import load_from_disk
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [2]:
def add_prompt_to_dataset(dataset, prompt_file="prompt.md"):
    with open(prompt_file, "r") as f:
        prompt = f.read()

    def prepare_prompt(batch):
        batch["prompt"] = [
            prompt.format(user_question=q, table_metadata_string=m)
            for q, m in zip(batch["question"], batch["context"])
        ]
        return batch

    dataset = dataset.map(prepare_prompt, batched=True)
    return dataset

In [3]:
def get_tokenizer_model(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        torch_dtype=torch.bfloat16, # May not work on All GPU, use float16 instead if error
        device_map="auto",
        use_cache=True,
    )
    return tokenizer, model

In [ ]:
# def generate_sql(input, tokenizer, model):
#     prompt = generate_prompt(input)
    
#     # make sure the model stops generating at triple ticks
#     # eos_token_id = tokenizer.convert_tokens_to_ids(["```"])[0]
#     eos_token_id = tokenizer.eos_token_id
#     pipe = pipeline(
#         "text-generation",
#         model=model,
#         tokenizer=tokenizer,
#         max_new_tokens=300,
#         do_sample=False,
#         return_full_text=False, # added return_full_text parameter to prevent splitting issues with prompt
#         num_beams=1, # do beam search with 4 beams for high quality results
#     )
#     generated_query = (
#         pipe(
#             prompt,
#             num_return_sequences=1,
#             eos_token_id=eos_token_id,
#             pad_token_id=eos_token_id,
#         )[0]["generated_text"]
#         .split(";")[0]
#         .split("```")[0]
#         .strip()
#     )
#     return generated_query

In [92]:
from tqdm import tqdm
from datasets import Dataset

class ListDataset(Dataset):
    def __init__(self, original_list):
        self.original_list = original_list

    def __len__(self):
        return len(self.original_list)

    def __getitem__(self, i):
        return self.original_list[i]


def generate_sql_batch(dataset, tokenizer, model, batch_size=16):
    dataset = add_prompt_to_dataset(dataset)
    dataset = ListDataset(dataset)

    eos_token_id = tokenizer.eos_token_id
    tokenizer.pad_token_id = eos_token_id
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=300,
        do_sample=False,
        return_full_text=False,
        num_beams=1,
    )

    all_generations = []

    for output in tqdm(
        pipe(
            dataset["prompt"],
            num_return_sequences=1,
            eos_token_id=eos_token_id,
            pad_token_id=eos_token_id,
            batch_size=batch_size,
        )
    ):
        generated_query = (
            output[0]["generated_text"].split(";")[0].split("```")[0].strip()
        )
        all_generations.append(generated_query)

    return all_generations

### Load

In [4]:
dataset = load_from_disk("./datasets/sql-create-context-split")
tokenizer, model = get_tokenizer_model("defog/sqlcoder-7b-2")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
test_dataset = dataset["train"]

### value for proper max token

In [8]:
# token_lengths = [
#     len(tokenizer.tokenize(answer)) for answer in dataset["train"]["answer"]
# ]

# # Analyze statistics
# max_length = max(token_lengths)
# mean_length = sum(token_lengths) / len(token_lengths)
# print(f"Max tokens: {max_length}, Mean tokens: {mean_length}")

300

In [6]:
token_lengths = [
    len(tokenizer.tokenize(prompt)) for prompt in add_prompt_to_dataset(dataset["train"])["prompt"]
]

# Analyze statistics
max_length = max(token_lengths)
mean_length = sum(token_lengths) / len(token_lengths)
print(f"Max tokens: {max_length}, Mean tokens: {mean_length}")

Map:   0%|          | 0/62861 [00:00<?, ? examples/s]

Max tokens: 407, Mean tokens: 158.44238876250776


### Run

In [ ]:
generations = generate_sql_batch(test_dataset, tokenizer, model, batch_size=8)

100%|██████████| 128/128 [00:00<00:00, 808540.53it/s]


In [ ]:
test_dataset = test_dataset.add_column("generation", generations)

In [91]:
test_dataset[123]

{'answer': 'SELECT COUNT(clock_rate__mhz_) FROM table_142573_1 WHERE bandwidth__mb_s_ = 2400',
 'question': 'Name the number of clock rate mhz when bandwidth mb/s is 2400',
 'context': 'CREATE TABLE table_142573_1 (clock_rate__mhz_ VARCHAR, bandwidth__mb_s_ VARCHAR)',
 'generation': "SELECT t.clock_rate__mhz_ FROM table_142573_1 t WHERE t.bandwidth__mb_s_ = '2400'"}

In [ ]:
def generate_prompt_batch(batch, prompt_file="prompt.md"):
    with open(prompt_file, "r") as f:
        prompt = f.read()

    prompt_batch = [
        prompt.format(user_question=q, table_metadata_string=m)
        for q, m in zip(batch["question"], batch["context"])
    ]

    return prompt_batch

In [ ]:
from tqdm import tqdm


def generate_sql_batch(dataset, tokenizer, model, batch_size=1):
    eos_token_id = tokenizer.eos_token_id
    tokenizer.pad_token_id = eos_token_id
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=300,
        do_sample=False,
        return_full_text=False,
        num_beams=1,
    )
    generations = []

    for i in tqdm(range(0, len(dataset), batch_size), desc="Generating:"):
        batch = dataset[i : i + batch_size]
        prompts = generate_prompt_batch(batch)

        outputs = pipe(
            prompts,
            num_return_sequences=1,
            eos_token_id=eos_token_id,
            pad_token_id=eos_token_id,
            batch_size=batch_size,
        )

        for output in outputs:
            generated_query = (
                output[0]["generated_text"].split(";")[0].split("```")[0].strip()
            )
            generations.append(generated_query)

    return generations

In [1]:
def make_prompts(dataset, prompt_file="prompt.md"):
    with open(prompt_file, "r") as f:
        prompt = f.read()

    prompt_batch = [
        prompt.format(user_question=q, table_metadata_string=m)
        for q, m in zip(dataset["question"], dataset["context"])
    ]

    return prompt_batch

In [3]:
from datasets import load_from_disk

dataset = load_from_disk("./datasets/sql-create-context-split")
data = dataset["val"]
data["question"]

['Show the locations of parties and the names of the party hosts in ascending order of the age of the host.',
 'List all customers’ names in the alphabetical order.',
 'Show all team names.',
 'List the name of all customers sorted by their account balance in ascending order.',
 "How many trains have 'Express' in their names?",
 'How many games are held after season 2007?',
 'What is the id and last name of the driver who participated in the most races after 2010?',
 'What is the name of the department that has the largest number of students enrolled?',
 'List all public schools and their locations.',
 'Show all headquarters with both a company in banking industry and a company in Oil and gas.',
 'How much is the track Fast As a Shark?',
 'Which allergy type has least number of allergies?',
 'Find the distinct driver id and the stop number of all drivers that have a shorter pit stop duration than some drivers in the race with id 841.',
 'List the names and emails of customers who payed